## Astronomy 406 "Computational Astrophysics" (Fall 2015)

### Week 10: Multimodality of a distribution, Expectation-Maximization, Model Selection

<b>Reading:</b> notes below, as well as $\S$16.1 of <a href="http://www.nr.com">Numerical Recipes</a>, and $\S$4.4, 6.3.1 of <a href="http://www.astroml.org/">Machine Learning</a>.

In [ ]:
%matplotlib inline
from matplotlib import rcParams
rcParams["savefig.dpi"] = 80
import numpy as np
import matplotlib.pyplot as plt

#### Calculating and interpreting confidence limits

In [ ]:
aa, bb, ss = np.loadtxt("bootstrap.dat", unpack=True)

def kde_tophat( data, x, h ):
    y = (abs(x - data[:,None]) <= h).astype(float)
    return y.sum(0)/(2*h*len(data))

def conf_limit( x, lim ):
    dx = np.sort(np.abs(x-np.mean(x)))
    return dx[int(lim*len(x))]

In [ ]:
b = np.mean(bb)
b1 = conf_limit(bb, 0.683)
b2 = conf_limit(bb, 0.954)
b3 = conf_limit(bb, 0.997)

print 'db: at 68.3 = %.3f  at 95.4 = %.3f  at 99.7 = %.3f' % (b1, b2, b3)

x = np.linspace(1.3, 1.6, 1000)
plt.plot(x, kde_tophat(bb, x, 0.02))
x = np.linspace(b-b1, b+b1, 200)
plt.fill_between(x, kde_tophat(bb, x, 0.02), 0, alpha=0.3)
plt.xlim(1.3, 1.57)
plt.xlabel('b')
plt.ylabel('N')

plt.plot([b, b], [0, 11], '--r')
plt.plot([b-b1, b-b1], [0, 7], '-k')
plt.plot([b+b1, b+b1], [0, 7.2], '-k')
plt.plot([b-b2, b-b2], [0, 1.7], '-k')
plt.plot([b+b2, b+b2], [0, 1.6], '-k')
plt.plot([b-b3, b-b3], [0, 0.3], '-k')
plt.plot([b+b3, b+b3], [0, 0.2], '-k')

### Gaussian Mixture Modeling

Let's split an observed distribution into a sum of Gaussian modes.  The probability of observing a point $x_n$ is

$P(x_n) = \sum_{k=1}^{K} p_k N(x_n | \mu_k,\sigma_k)$

where $p_k$ are the weights of each mode $k$.  The likelihood of observing the whole data set is

${\cal L} = \prod_n P(x_n)$.

In [ ]:
x = np.loadtxt("DataFiles/gc.dat", usecols=(0,), unpack=True)

In [ ]:
mu = [-1, 0]
sig = [1., 1.]
pk = [0.5, 0.5]

Kmodes = len(mu)
logL = 999.
logL1 = 99.
iter = 0
pnk = np.zeros([Kmodes, len(x)])

while abs(logL-logL1) > 1.e-6:
    # E-step
    iter += 1
    pxn = np.zeros(len(x))
    for k in xrange(Kmodes):
        pxn += pk[k]/np.sqrt(2.*np.pi)/sig[k]*np.exp(-((x-mu[k])/sig[k])**2/2.)

    logL1 = logL
    logL = np.sum(np.log(pxn))

    # M-step
    for k in xrange(Kmodes):     
        pnk[k] = pk[k]/np.sqrt(2.*np.pi)/sig[k]*np.exp(-((x-mu[k])/sig[k])**2/2.)/pxn

    for k in xrange(Kmodes):
        w = pnk[k]/sig[k]**2
        sig2k = np.average((x-mu[k])**2, weights=w)
        sig[k] = np.sqrt(sig2k)
        mu[k] = np.average(x, weights=w)
        pk[k] = np.average(pnk[k])
        
print logL

In [ ]:
d = 0.1
xmin = -2.5
xmax = 0.5
bins = np.arange(xmin, xmax, d)
xx = np.linspace(xmin, xmax, num=100)
yy = np.zeros(len(xx))
for k in xrange(len(mu)):
    yy += len(x)*d*pk[k]/np.sqrt(2.*np.pi)/sig[k]\
          *np.exp(-((xx-mu[k])/sig[k])**2/2.)    
        
plt.xlabel('x')
plt.ylabel('N')
plt.hist(x, bins)
plt.plot(xx, yy, 'r-')

How do we choose the right number of modes to describe our data set?  It is a non-trivial problem, which is a part of general problem of <b>model selection</b>.  We consider some simple rules first.

The Aikake information criterion (derived from information theory):

$\mathrm{AIC} \equiv -2\ln{{\cal L}_{max}} + 2K + {2K (K+1) \over N-K-1}$.

The Bayesian information criterion:

$\mathrm{BIC} \equiv -2\ln{{\cal L}_{max}} + K \ln{N}$.

In principle, the likelihood ${\cal L}$ can always be improved by increasing the number of free parameters.  Both criteria penalize the likelihood for number of model parameters $K$.  The most appropriate model would minimize AIC and BIC.

<b>Exercise 1:</b> Choose the right number of modes for our data set based on AIC and BIC.

In [ ]:
def GMM_simple(mu, sig, pk, x):
    ...
    return logL

We will use the GMM algorithm provided by <b>sklearn</b> package and take examples of <a href="http://www.astroml.org/book_figures/chapter4/fig_GMM_1D.html">Figure 4.2</a> and <a href="http://www.astroml.org/book_figures/chapter6/fig_EM_metallicity.html">Figure 6.6</a> in ML.

Read also <a href="http://scikit-learn.org/stable/modules/mixture.html">useful information</a> on the implementation of GMM in sklearn.

In [ ]:
from sklearn.mixture import GMM
np.random.seed(0)

In [ ]:
K = np.arange(1, 6)
models = [None for i in K]

X = x.reshape(-1,1)

for i in range(len(K)):
    models[i] = GMM(K[i], random_state=1, n_init=10).fit(X)

AIC = [m.aic(X) for m in models]
BIC = [m.bic(X) for m in models]
print "AIC=", AIC[:4]
print "BIC=", BIC[:4]

M_best = models[np.argmin(AIC)]

d = 0.2
xmin = -2.5
xmax = 0.5
bins = np.arange(xmin, xmax, d)
xx = np.linspace(xmin, xmax, num=100)
yy = np.zeros(len(xx))

logprob = M_best.score_samples(X)
pdf = np.exp(logprob[:][0])
pdf_individual = responsibilities * pdf[:, np.newaxis]

plt.hist(x, bins, normed=True, histtype='stepfilled', alpha=0.4)
plt.plot(xx, pdf, '-k')
plt.plot(xx, pdf_individual, '--k')
plt.xlabel('x')
plt.ylabel('p(x)')

In [ ]:
plt.plot(K, AIC, 'r-', label='AIC')
plt.plot(K, BIC, 'b--', label='BIC')
plt.xlabel('number of components')
plt.ylabel('information criterion')
plt.legend(loc=2, frameon=False)

In [ ]:
feh, m, d = np.loadtxt("DataFiles/gc.dat", unpack=True)
logd = np.log10(d)
X = np.vstack([feh, logd]).T

In [ ]:
N = np.arange(1, 10)
models = [None for n in N]

for i in range(len(N)):
    models[i] = GMM(N[i], covariance_type='full').fit(X)

AIC = [m.aic(X) for m in models]
BIC = [m.bic(X) for m in models]

i_best = np.argmin(BIC)
gmm_best = models[i_best]
print "best fit converged:", gmm_best.converged_
print "BIC: N components = %i" % N[i_best]

plt.plot(N, AIC, 'r-', label='AIC')
plt.plot(N, BIC, 'b--', label='BIC')
plt.xlabel('number of components')
plt.ylabel('information criterion')
plt.legend(loc=2, frameon=False)

In [ ]:
from astroML.plotting.tools import draw_ellipse

plt.scatter(feh, logd)
plt.xlabel('[Fe/H]')
plt.ylabel('log d (kpc)')

for mu, C, w in zip(gmm_best.means_, gmm_best.covars_, gmm_best.weights_):
    draw_ellipse(mu, C, scales=[2], fc='none', ec='k')

<b>Exercise 2:</b> Compare the GMM model parameters and information criteria obtained with the explicit method above and with the sklearn package, for the univariable dataset feh.